In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
# ****************************************************************
# ****************************************************************
# ****************************************************************

pOoiInitial = 'laptop'
pOoi   = 'laptop'
pOoiPr = '69'
cOoi   = 'waterBottle'
cOoiPr = '23'

# ****************************************************************
# ****************************************************************
# ****************************************************************
# 
# 
# # File Two: Read the sorts, build the events and store in a .txt file.


import fileinput
import sys

def initialFluent(file, searchExp, replaceExp):
    #https://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

def timepoint():
    happensCount = open('./lp/events.lp')
    lines = happensCount.readlines()
    print(lines[-1])
    start = '), '
    end = ').'
    extracted = lines[-1][lines[-1].find(start)+len(start):lines[-1].rfind(end)]
    # time = int((str(lines).count('happens(')))
    time = str(int(extracted)+1)
    return time

def event(pOoi, cOoi, time):
    pEChanOOI = 'happens(ooiChange(' + pOoi + ',' + ' ' + cOoi + '), ' + time + ').'
    print (pEChanOOI)
    with open('./lp/events.lp', 'a') as out:
        out.write('\n{}'.format(pEChanOOI))
        print ('./lp/events.lp is stored')

In [ ]:
# Test cell

file = './lp/events.lp'
searchExp  = 'holdsAt(oOI(waterBottle'
replaceExp = 'holdsAt(oOI(' + pOoiInitial

initialFluent(file, searchExp, replaceExp)

pOoi = 'cup'
cOoi = 'sugar'

# happens(ooiChange(laptop, waterBottle), 0).
# happens(ooiChange(waterBottle, spoon), 4).
# happens(ooiChange(spoon, cup), 5).
# happens(ooiChange(cup, sugar), 6).
# happens(ooiChange(sugar, teapot), 7).

time = timepoint()
event(pOoi, cOoi, time)

In [ ]:
# Run the reasoning system.
import os
import subprocess

time = timepoint()

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'

clingoTime = int(time) + 2
clingoTime = str(clingoTime)

!chmod +x ./clingo/clingoLinux

"""
command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " + \
            "lp\\decOrigin2.lp " + \
            "lp\\sorts.lp " + \
            "lp\\events.lp " + \
            "lp\\activities.lp " + \
            "lp\\ecasp888.lp " + \
            "lp\\ecasp8882.lp " + \
            "lp\\ecasp88888.lp | " + \
            "clingo\\format-output.exe 3 > " + \
            "inference\\ecasp88.txt"""
            
command = mainPath + 'clingo/clingoLinux -c maxtime=' + clingoTime + ' -n 1 ' + \
            mainPath + 'lp/decOrigin2.lp ' + \
            mainPath + 'lp/sorts.lp ' + \
            mainPath + 'lp/events.lp ' + \
            mainPath + 'lp/activities.lp ' + \
            mainPath + 'lp/ecasp888.lp ' + \
            mainPath + 'lp/ecasp8882.lp ' + \
            mainPath + 'lp/ecasp88888.lp | ' + \
            mainPath + 'clingo/format-output13042022_4.bin ' + clingoTime + ' > ' + \
            mainPath + 'inference/ecasp88.txt'

# print(command)
osCommand = os.system(command)

In [ ]:
# Read the atomic activities and their respective timepoints
## Read answer set extracts by timepoints

import re

def answerSet(pathToAnswerSet):
    text_file = open(pathToAnswerSet, "r")
    data = text_file.read()
    text_file.close()

    regex = r"((?:[^\n][\n]?)+)"

    matches = re.finditer(regex, data)

    grouped = []

    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            grouped.append(match.group(groupNum))
    return grouped

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'        
file = "inference/ecasp88.txt"

grouped = answerSet(mainPath + file)

## Read the atomic activities from the answer set extracts

def findAtomicActivities(fromGrouped):
    start = 'Activity('
    end = ')'
    extracted = fromGrouped[fromGrouped.find(start)+len(start):fromGrouped.rfind(end)]
    previous, current, probability = extracted.split(',')
    return previous, current, probability

timepoints = []
activity   = []
object     = []
probabilit = []

match = "+atomicActivity("

for ff, gg in enumerate(grouped):
    m = re.search('\+atomicActivity\((.+?),.+\)', grouped[ff])
    if m:
        found = m.group(0)
        previous, current, probability = findAtomicActivities(found)
        timepoints.append(grouped[ff][0])
        activity.append(previous)
        object.append(current)
        probabilit.append(probability)

print(timepoints)
print(activity)
print(object)
print(probabilit)

In [ ]:
# Read the composite activity predicate and extract the atomic activities

def atomicActOntology(ontology):
    start = 'Activity('
    end = ')'
    extracted = ontology[ontology.find(start)+len(start):ontology.rfind(end)]
    atomic = extracted.split(', ')
    return atomic

fileOntology = "lp/ecasp88888.lp"
ontology = answerSet(mainPath + fileOntology)

numberOfAtomic = ontology[3].count(',')
atomic = atomicActOntology(ontology[3])
print(atomic)

TT = []

# for hh in range(numberOfAtomic):
#     # print(atomic[hh])
#     # print(activity[0])
#     # if atomic[hh] in activity:
#     TT.append(timepoints[activity.index(atomic[hh])])
#     print(TT)

res = []
i = 0
while (i < len(activity)):
    if (atomic.count(atomic[i]) > 0):
        res.append(i)
    i += 1
print(res)
  
# # printing result 
# print("The matching element Indices list : " + str(res))

In [ ]:
# Read file ecasp8882.lp

T = TT[3]
T1 = TT[2]
T2 = TT[1]
T3 = TT[0]

composite = "happens(compositeChange(A1, A2, A3, A4), " + str(T) + ") :- \n \
				holdsAt(atomicActivity(A4, O4, Pa4), " + str(T) + "), \n \
				holdsAt(atomicActivity(A3, O3, Pa3), " + str(T1) + "), \n \
				holdsAt(atomicActivity(A2, O2, Pa2), " + str(T2) + "), \n \
				holdsAt(atomicActivity(A1, O1, Pa1), " + str(T3) + "), \n \
				contextCompositeActivity(A1, A2, A3, A4, A5), \n \
				objectOI(O1, Po1), objectOI(O2, Po2), \n \
				objectOI(O3, Po3), objectOI(O4, Po4),\n \
				affordance(A1), affordance(A2), affordance(A3),\n \
				affordance(A4), affordance(A5),	time(T)."

with open('./lp/ecasp8882.lp', 'w') as out:
	out.write(composite)
	print ('./lp/ecasp8882.lp is stored')

In [ ]:
# Problem in matching the elements lists of the Answer Set and those of the ontology
# to perform the timepoints extractions to add them in the previous cell for inferring
# the composite activity.

In [ ]:
import re

atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

print(line)

for i in range(len(line)):
    # atomicActivity = re.search('\+atomicActivity\((.+?),', line[i])
    atomicActivity = re.search('((?:[^\n][\n]?)+)', line[i])
    print(atomicActivity, i)

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

#open text file in read mode
text_file = open(mainPath + "inference/ecasp88.txt", "r")
 
#read whole file to a string
data = text_file.read()
 
#close file
text_file.close()
 
print(data)

line = str(line)
# line = line.replace("',","")
# line = line.replace("'","")
print(line)

regex = r"((?:[^\n][\n]?)+)"

test_str = ("\n"
	"==========\n"
	"Answer: 1\n\n"
	"0\n"
	"happens(atomicChange(waterBottle,office),0)\n"
	"happens(ooiChange(laptop,waterBottle),0)\n"
	"oOI(laptop,70)\n"
	"currentLocation(office,56)\n\n"
	"1\n"
	"-oOI(laptop,70)\n"
	"+atomicActivity(serve,waterBottle,49)\n"
	"currentLocation(office,56)\n"
	"happens(atomicChange(spoon,office),1)\n"
	"happens(ooiChange(waterBottle,spoon),1)")

matches = re.finditer(regex, data)

grouped = []

for matchNum, match in enumerate(matches, start=1):
    # print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        grouped.append(match.group(groupNum))
        # print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
text_file = open(mainPath + "inference/ecasp88.txt", "r")
data = text_file.read()
text_file.close()

regex = r"((?:[^\n][\n]?)+)"

matches = re.finditer(regex, data)

grouped = []

for matchNum, match in enumerate(matches, start=1):
    # print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        grouped.append(match.group(groupNum))
        # print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
print(grouped[5])

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

#line = str(line)
# print(line)

spaces = []

for s in range(len(line)):
	if line[s] == '\n':
		spaces.append(s)

print(spaces)

for ss in range(len(spaces)):
	# print(ss)
	# print(type(ss))
    # print(line[ss])
    # print(line[ss+1])
	for rr in range(spaces[ss],spaces[ss+1],1):
		print(line[rr])

In [ ]:
# File One: Read the sorts and store in a .txt file.


# ****************************************************************
# ****************************************************************
# ****************************************************************

pLoc   = 'office'
pLocPr = '78'
cLoc   = 'desktop'
cLocPr = '65'

pOoi   = 'cup'
pOoiPr = '69'
cOoi   = 'chair'
cOoiPr = '23'

# ****************************************************************
# ****************************************************************
# ****************************************************************

pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'

pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'

print (pLocation)
print (cLocation)
print (pObjectOI)
print (cObjectOI)

with open('./lp/sorts.lp', 'w') as out:
    out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
    print ('sorts.lp is stored')

# File Two: Read the sorts, build the events and store in a .txt file.


pEChanLoc = 'happens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'

pEChanOOI = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

print (pEChanLoc)
print (pEChanOOI)

csloi = "not releasedAt(F,0) :- fluent(F)."

with open('./lp/events.lp', 'w') as out:
    out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
    print ('./lp/events.lp is stored')

In [ ]:
# Run the reasoning system.
import os

command = "./clingo/clingoWindows.exe -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output.exe 3 >./inference/ecasp88.txt"
# command = "./clingo/clingoLinux -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output 3 >./inference/ecasp88.txt"

os.system(command)

In [ ]:
# Read the intention.

intention = open("./inference/ecasp88.txt")
line = intention.readlines()
print (line[15])

match = "+intention("

if match in line[15]:
    print ("Yes, it is. Hihi!!")

In [ ]:
# Extract the intention.

import re

m = re.search('\+intention\((.+?),', line[15])
if m:
    found = m.group(1)

In [ ]:
found

In [ ]:
# Resize video

import cv2
import numpy as np

pathToResize = mainPath + "input/"

print(pathToResize + "Egocentric18022022_3C_2_CASE2022_detected.mp4")

# cap = cv2.VideoCapture(pathToResize + "Robot17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_detected.mp4")
cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_CASE2022_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_1FPS_detected.mp4")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(pathToResize + "Robot17022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_detectedResized.mp4",fourcc, 3.0, (1920,1080))
out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_CASE2022_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(1920,1080))
        out.write(b)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
# Reduce FPS

import cv2

# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot17022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot18022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot18022022_2C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/YDXJ0133.mp4"
pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/kitchen4.mp4"

# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric17022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric18022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric18022022_2C.mp4"

# pathOutputFrames = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputEgocentric/frame"
pathOutputFrames = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/output/frames/intentionICTAI06072022"

vidcap = cv2.VideoCapture(pathToReduce)

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000) 
    hasFrames,image = vidcap.read() 
    if hasFrames:
        cv2.imwrite(pathOutputFrames + str(sec) + ".jpg", image)
    return hasFrames 

sec = 0
frameRate = 1 #it will capture image in each 0.5 second
success = getFrame(sec)
while success:
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
# oilBottle         0
# waterBottle       1
# bowl              2
# salade            3
# tap               4
# coffeeMachine     5
# milkBottle        6
# salt              7
# dishwashLiquid    8
# sponge            9
# vase             10
# microwave        11
# coffeeBox        12
# lhand            13
# rhand            14
# box              15
# cup              16
# plate            17
# coffeeCapsule    18
# fork             19


# oilBottle         0              87
# waterBottle       1              83
# bowl              2              45
# salade            3              92              *
# tap               4              93              *
# coffeeMachine     5              94              *
# milkBottle        6              95              *
# salt              7              96              *
# dishwashLiquid    8              91
# sponge            9              97              *
# vase             10              75
# microwave        11              68
# coffeeBox        12              98              *
# lhand            13              81
# rhand            14              80
# box              15              99              *
# cup              16              41
# plate            17             100              *
# coffeeCapsule    18             101              *
# fork             19              42

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","87 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","83 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","45 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","92 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","93 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","94 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","95 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","96 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","91 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","97 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","75 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","68 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","98 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","81 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","80 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","99 ") + line[3:]
        elif "16 " in line[0:3]:
            line = line[0:3].replace("16 ","41 ") + line[3:]
        elif "17 " in line[0:3]:
            line = line[0:3].replace("17 ","100 ") + line[3:]
        elif "18 " in line[0:3]:
            line = line[0:3].replace("18 ","101 ") + line[3:]
        elif "19 " in line[0:3]:
            line = line[0:3].replace("19 ","42 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(0, 73.0, 0.5)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# dishwashLiquid    0
# oilBottle         1
# tap               2
# plate             3
# salt              4
# coffeeBox         5
# milkBottle        6
# bowl              7
# lhand             8
# rhand             9
# sponge           10
# fork             11
# microwave        12
# vase             13
# waterBottle      14
# salade           15
# coffeeMachine    16
# box              17
# cup              18
# coffeeCapsule    19


# dishwashLiquid    0              91
# oilBottle         1              87
# tap               2              93              *
# plate             3             100              *
# salt              4              96              *
# coffeeBox         5              98              *
# milkBottle        6              95              *
# bowl              7              45
# lhand             8              81
# rhand             9              80
# sponge           10              97              *
# fork             11              42
# microwave        12              68
# vase             13              75
# waterBottle      14              83
# salade           15              92              *
# coffeeMachine    16              94              *
# box              17              99              *
# cup              18              41
# coffeeCapsule    19             101              *

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","91 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","87 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","93 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","100 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","96 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","98 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","95 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","45 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","81 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","80 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","97 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","42 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","68 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","75 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","83 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","92 ") + line[3:]
        elif "16 " in line[0:3]:
            line = line[0:3].replace("16 ","94 ") + line[3:]
        elif "17 " in line[0:3]:
            line = line[0:3].replace("17 ","99 ") + line[3:]
        elif "18 " in line[0:3]:
            line = line[0:3].replace("18 ","41 ") + line[3:]
        elif "19 " in line[0:3]:
            line = line[0:3].replace("19 ","101 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(73, 130, 0.5)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# lhand             0
# rhand             1
# plate             2
# milkBottle        3
# coffeeBox         4
# bowl              5
# tap               6
# salt              7
# dishwashLiquid    8
# sponge            9
# oilBottle        10
# box              11
# fork             12
# microwave        13
# vase             14
# waterBottle      15
# salade           16
# coffeeMachine    17
# cup              18

# lhand             0              81
# rhand             1              80
# plate             2             100              *
# milkBottle        3              95              *
# coffeeBox         4              98              *
# bowl              5              45
# tap               6              93              *
# salt              7              96              *
# dishwashLiquid    8              91
# sponge            9              97              *
# oilBottle        10              87
# box              11              99              *
# fork             12              42
# microwave        13              68
# vase             14              75
# waterBottle      15              83
# salade           16              92              *
# coffeeMachine    17              94              *
# cup              18              41

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","81 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","80 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","100 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","95 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","98 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","45 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","93 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","96 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","91 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","97 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","87 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","99 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","42 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","68 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","75 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","83 ") + line[3:]
        elif "16 " in line[0:3]:
            line = line[0:3].replace("16 ","92 ") + line[3:]
        elif "17 " in line[0:3]:
            line = line[0:3].replace("17 ","94 ") + line[3:]
        elif "18 " in line[0:3]:
            line = line[0:3].replace("18 ","41 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(130, 293, 0.5)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# AnticipationEgocentric17022022

# rhand             0              80
# lhand             1              81
# laptop            2              63
# robot             3              82
# mouse             4              64
# cup               5              41
# apple             6              47
# water bottle      7              83
# cell phone        8              67
# book              9              73
# pen              10              84
# banana           11              46

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","80 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","81 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","63 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","82 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","64 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","41 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","47 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","83 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","67 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","73 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","84 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","46 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(range(95)):
    file = "/raid/home/labuseraber/environment/testEnvironment/reformat/output/frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# AnticipationEgocentric04022022

# robot             0              82
# phone             1              67
# paper             2             102 
# mouse             3              64
# laptop            4              63
# cup               5              41
# pen               6              84
# banana            7              46
# rhand             8              80
# lhand             9              81

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","82 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","67 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","102 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","64 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","63 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","41 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","84 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","46 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","80 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","81 ") + line[2:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(range(84)):
    file = "/raid/home/labuseraber/environment/testEnvironment/reformat/AnticipationEgocentric04022022/frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# Rename files

import os
import numpy as np
from tqdm import tqdm

for i in tqdm(range(102)):
    framesName = "C:\\Users\\Administrator\\Downloads\\originLabeling\\anticipationCocoEgocentric17022022\\inputRenamed\\frame" + str(i) + ".jpg"
    labelsName = "C:\\Users\\Administrator\\Downloads\\originLabeling\\anticipationCocoEgocentric17022022\\outputRenamed\\frame" + str(i) + ".txt"
    
    framesNameRenamed = "C:\\Users\\Administrator\\Downloads\\originLabeling\\anticipationCocoEgocentric17022022\\inputRenamed\\anticipationCocoEgocentric17022022" + str(i) + ".jpg"
    labelsNameRenamed = "C:\\Users\\Administrator\\Downloads\\originLabeling\\anticipationCocoEgocentric17022022\\outputRenamed\\anticipationCocoEgocentric17022022" + str(i) + ".txt"

    os.rename(labelsName, labelsNameRenamed)
    os.rename(framesName, framesNameRenamed)

In [ ]:
import cv2

vidcap = cv2.VideoCapture('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/kitchen4.mp4')
fps = vidcap.get(cv2.CAP_PROP_FPS)
success,image = vidcap.read()
count = 0
success = True

while success:
    success,frame = vidcap.read()
    count+=1
    print("time stamp current frame:",count/fps)

In [ ]:
import cv2

video_name = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/kitchen4.mp4"
vidcap = cv2.VideoCapture(video_name)
success, image = vidcap.read()
count = 0

while success:
    cv2.imwrite("frame%d.jpg" % count, image)
    success, image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

In [ ]:
import json
from tqdm import tqdm
import cv2
import json
import numpy as np

radius = 20
color = (0, 255, 0)
thickness = 2

video = "/raid/home/labuseraber/environment/testEnvironment/yolov5/runs/detect/exp7/kitchen4.mp4"
vidcap = cv2.VideoCapture(video)
success, img = vidcap.read()
height, width, channels = img.shape
count = 0

# while success:
#     cv2.imwrite("frame%d.jpg" % count, image)
#     success, image = vidcap.read()
#     print('Read a new frame: ', success)
#     count += 1

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('./kitchen4Gaze.mp4', fourcc, 25, (width, height))

r = open('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/gaze.json')
line = r.readlines()

for i in tqdm(np.arange(0, 22068, 4)):
    e = json.loads(line[i])
    if e["data"]:
        x = e["data"]["gaze2d"][0]
        y = e["data"]["gaze2d"][1]
        img = cv2.circle(img, (int(x*width), int(y*height)), radius, color, thickness)
        video.write(img)
    else:
        video.write(img)

In [ ]:
import json
from tqdm import tqdm
import cv2
import json
import numpy as np
from opencv_jupyter_ui import cv2_imshow

radius = 20
color = (255, 255, 255)
thickness = 2

vid = "/raid/home/labuseraber/environment/testEnvironment/yolov5/runs/detect/exp7/kitchen4.mp4"
vidcap = cv2.VideoCapture(vid)
success, img = vidcap.read()
height, width, channels = img.shape
count = 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('./kitchen4Gaze2.mp4', fourcc, 25, (width, height))

def read(lineNumber):
    r = open('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/gaze.json')
    line = r.readlines()
    e = json.loads(line[lineNumber])
    # if e["data"]:
    #     x = e["data"]["gaze2d"][0]
    #     y = e["data"]["gaze2d"][1]
    #     img = cv2.circle(img, (int(x*width), int(y*height)), radius, color, thickness)
    #     video.write(img)
    # else:
    #     video.write(img)
    # return x, y
    if e["data"]:
        x = e["data"]["gaze2d"][0]
        y = e["data"]["gaze2d"][1]
        return x, y
    else:
        x=0
        y=0
        return x, y

for i in tqdm(range(5516)):
    success, img = vidcap.read()
    count = count + 4
    # print(i, count)
    x, y = read(count)
    img = cv2.circle(img, (int(x*width), int(y*height)), radius, color, thickness)
    # cv2_imshow('test',img)
    video.write(img)

cv2.destroyAllWindows()
video.release()

In [ ]:
import json

def read(lineNumber):
    r = open('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/gaze.json')
    line = r.readlines()
    e = json.loads(line[lineNumber])
    if e["data"]:
        x = e["data"]["gaze2d"][0]
        y = e["data"]["gaze2d"][1]
        return x, y
    else:
        x=0
        y=0
        return x, y

In [ ]:
x, y = read(1)
print(x,y)